In [ ]:
pip install transformers
pip install sentence_transformers
pip install scipy

In [ ]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Importing the data file and the data was the validation split of training data

In [ ]:
# Load the File
df_str_rel = pd.read_csv('/content/drive/MyDrive/val.csv',on_bad_lines='skip')
df_str_rel.head()

,index,Text1,Text2,Type,Lang,Scores,e5_mean,e5_trained,Labels,Labels_without_training,Labels_with_training,Unsupervised,Gold Lable,cosine_conti,cosine_discrete
0,6913,ይህ ደግሞ ቀላል አይሆንም ።,ያ ደግሞ ቀላል አይደለም ።,train,amh,0.75,0.995844,0.980574,1,1,1,1,1,0.981341,1
1,6806,መንገድ መሪ ከሆነ ማንኛውም ሰው የሚፈለገው ዋነኛ ባሕርይ እምነት የሚጣል...,የበረራ አዲስ መረጃ ፡ ኢትዮጵያን አየርመንገድ አሁንድ ኢስታንቡል እየበረ...,train,amh,0.19,0.805951,0.320204,0,1,0,0,0,0.762803,1
2,7283,ነገ ሰኔ 16 ነው ፡፡,ሰኔ 16/17 ትምህርት ቤት ቢባል ይሻላል :-) !,train,amh,0.36,0.898698,0.751506,0,1,1,1,0,0.876315,1
3,7578,አንድ ረጅም ልቦለድ እየፃፍኩ ነው ፡፡,ይህ መጽሐፍ ልቦለድ ነው ፡፡,train,amh,0.55,0.913872,0.780404,1,1,1,1,1,0.875937,1
4,6939,ሌላም ችግር አለ ነው ያልኩት ::,መቼ የት ነው እንዲህ ያልኩት ?,train,amh,0.30,0.908220,0.451932,0,1,0,0,0,0.845621,1


Thresholding the true lable at 0.5 above as 1 else 0 and storing it to data file with column name as Gold Lable

In [ ]:
gold_lable=[]
for t in df_str_rel['Scores']:
  if(t>0.5):
    gold_lable.append(1)
  else:
    gold_lable.append(0)

In [ ]:
df_str_rel['Gold Lable']=gold_lable
df_str_rel.to_csv('/content/drive/MyDrive/val.csv', index=False)

Extracting text from the data file as two input sentences list

In [ ]:
input1=[]
input2=[]
for index,row in df_str_rel.iterrows():
  s1= row["Text1"]
  s2= row["Text2"]
  input1.append(s1)
  input2.append(s2)
len(input1)

1160

Creating cluster by DBSCAN and thresholding as 1 if belong to same cluster else 0 then adding this column in data file

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/multilingual-e5-large')

In [ ]:
pred_lable=[]
for i in range(len(input1)):
  embeddings = model.encode([input1[i],input2[i]], normalize_embeddings=True)
  clustering = DBSCAN(eps=0.5, min_samples=2).fit(embeddings)
  if((clustering.labels_)[0]==0):
    pred_lable.append(1)
  else:
    pred_lable.append(0)

In [ ]:
df_str_rel['Unsupervised'] =pred_lable
df_str_rel.to_csv('/content/drive/MyDrive/val.csv', index=False)

calculating the accuracy score between unsupervised discrete score and true discrete score

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_str_rel['Unsupervised'], df_str_rel['Gold Lable'])

0.7827586206896552

calculating the accuracy score between unsupervised discrete score and true discrete score for all lang seprately

In [ ]:
accuracy_for_each_lang=[]
for lang, lang_df in df_str_rel.groupby(by='Lang'):
  accuracy_for_each_lang.append(accuracy_score(lang_df['Gold Lable'], lang_df['Unsupervised']))

In [ ]:
print(accuracy_for_each_lang)

[0.7070707070707071, 0.58, 0.8327272727272728, 0.7241379310344828, 0.8, 0.8547008547008547]


Storing the embeddings for all sentence pairs

In [ ]:
import tensorflow as tf
a = np.zeros(shape=(1160, 2, 1024))
for i in range(len(input1)):
  embeddings = model.encode([input1[i],input2[i]], normalize_embeddings=True)
  a[i] = embeddings

calculating the cosine similarity between two sentence pair embedding and storing it to data file with column name as

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
cosine_pred_lable=[]
# Example embeddings (1-D arrays)
for i in a:
  embedding1 = i[0]
  embedding2 = i[1]

# Reshape the embeddings to 2-D arrays to match the input format of cosine_similarity
  embedding1 = embedding1.reshape(1, -1)
  embedding2 = embedding2.reshape(1, -1)

# Compute the cosine similarity
  cosine_sim = cosine_similarity(embedding1, embedding2)
  cosine_pred_lable.append(cosine_sim[0][0])



In [ ]:
df_str_rel['cosine_conti']=cosine_pred_lable
df_str_rel.to_csv('/content/drive/MyDrive/val.csv', index=False)

Thresholding the cosine similarity score into discrete lable 0 and 1 and storing into data file with column name as cosine_discrete

In [ ]:
cosine_pred_lable1=[]
for t in df_str_rel['cosine_conti']:
  if(t>0.5):
    cosine_pred_lable1.append(1)
  else:
    cosine_pred_lable1.append(0)

In [ ]:
df_str_rel['cosine_discrete']=cosine_pred_lable1
df_str_rel.to_csv('/content/drive/MyDrive/val.csv', index=False)

calculating acurracy score between cosine discrete and gold lable discrete

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_str_rel['cosine_discrete'], df_str_rel['Gold Lable'])

0.4387931034482759

calculating spearman caorrelation coffecient between cosine continues score and continues true score

In [ ]:
import scipy
print(scipy.stats.spearmanr(df_str_rel['cosine_conti'],df_str_rel['Scores'])[0])

0.7112934980709242


In [ ]:
df_str_rel.head()

,index,Text1,Text2,Type,Lang,Scores,e5_mean,e5_trained,Labels,Labels_without_training,Labels_with_training,Unsupervised,Gold Lable,cosine_conti,cosine_discrete
0,6913,ይህ ደግሞ ቀላል አይሆንም ።,ያ ደግሞ ቀላል አይደለም ።,train,amh,0.75,0.995844,0.980574,1,1,1,1,1,0.981341,1
1,6806,መንገድ መሪ ከሆነ ማንኛውም ሰው የሚፈለገው ዋነኛ ባሕርይ እምነት የሚጣል...,የበረራ አዲስ መረጃ ፡ ኢትዮጵያን አየርመንገድ አሁንድ ኢስታንቡል እየበረ...,train,amh,0.19,0.805951,0.320204,0,1,0,0,0,0.762803,1
2,7283,ነገ ሰኔ 16 ነው ፡፡,ሰኔ 16/17 ትምህርት ቤት ቢባል ይሻላል :-) !,train,amh,0.36,0.898698,0.751506,0,1,1,1,0,0.876315,1
3,7578,አንድ ረጅም ልቦለድ እየፃፍኩ ነው ፡፡,ይህ መጽሐፍ ልቦለድ ነው ፡፡,train,amh,0.55,0.913872,0.780404,1,1,1,1,1,0.875937,1
4,6939,ሌላም ችግር አለ ነው ያልኩት ::,መቼ የት ነው እንዲህ ያልኩት ?,train,amh,0.30,0.908220,0.451932,0,1,0,0,0,0.845621,1
